<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-self-study/blob/master/5-2%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98(%EC%9E%91%EC%97%85%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 데이터 준비

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
print(wine)

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
print(data, target)

      alcohol  sugar    pH  class
0         9.4    1.9  3.51    0.0
1         9.8    2.6  3.20    0.0
2         9.8    2.3  3.26    0.0
3         9.8    1.9  3.16    0.0
4         9.4    1.9  3.51    0.0
...       ...    ...   ...    ...
6492     11.2    1.6  3.27    1.0
6493      9.6    8.0  3.15    1.0
6494      9.4    1.2  2.99    1.0
6495     12.8    1.1  3.34    1.0
6496     11.8    0.8  3.26    1.0

[6497 rows x 4 columns]
[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]] [0. 0. 0. ... 1. 1. 1.]


In [2]:
# 훈련 세트와 테스트 세트로 나눔.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


## 검증 세트

- 현재까지는 훈련 세트로 모델을 훈련하고 테스트 세트로 모델을 평가했음. 그러나 테스트 세트만으로 계속 성능을 확인하다 보면 점점 테스트 세트에 일반화 성능을 맞추게 되는 셈임.

- 테스트 세트로 일반화 성능을 올바르게 예측하려면 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋음.

- 그러나 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어려움. 테스트 세트를 사용하지 않고 측정하는 방법은 훈련 세트를 또 나누는 것임. 이 데이터를 '검증 세트(validation set)' 라고 부름. 실제로 많이 사용하는 방법임.

 1. 훈련 세트 80%, 테스트 세트 20%로 나누고 훈련 세트에서 다시 20%를 떼서 검증 세트로 만드는 식임. 보통 테스트 세트와 검증 세트를 20 ~ 30%만큼 떼지만, 훈련 세트가 아주 많다면 몇 %만 떼어 놓아도 전체 데이터를 대표하는 데 문제가 없음.

 1. 훈련 세트로 모델을 훈련한 후 검증 세트로 모델을 평가함. 이런 방식으로 테스트 하고자 하는 매개변수를 바꿔가며 가장 좋은 모델을 고름. 그리고 나서 이 매개변수를 사용하여 훈련 세트와 검증 세트를 합친 전체 훈련 데이터로 모델을 다시 훈련함.

 1. 마지막에 테스트 세트로 최종 점수를 평가함. 실전에서 테스트 세트 점수와 비슷한 성능을 기대할 수 있음.

### 검증 세트 만들기

In [3]:
# train_input과 train_target을 다시 train_test_split() 함수에 넣어서,
# 훈련 세트를 sub_input, sub_target으로 만들고,
# 검증 세트를 val_input, val_target으로 만듦.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)
print(sub_input.shape, val_input.shape)
# 훈련 세트의 샘플 5197개를 나눠서 훈련 세트 80%와 검증 세트 20%로 만들었음.

(4157, 3) (1040, 3)


### 결정 트리 모델 만들기

In [4]:
# 훈련 세트와 검증 세트로 결정 트리 모델을 훈련함.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 과대적합 된 상태이므로 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

0.9971133028626413
0.864423076923077


## 교차 검증

- 검증 세트를 만드느라 훈련 세트의 총량이 줄었음. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지지만, 그렇다고 검증 세트를 너무 조금 떼면 검증 점수가 불안정할 것임. 이런 경우에 '교차 검증(cross validation)' 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음.

 1. 검증 세트를 떼서 평가하는 과정을 여러번 반복함.
 
 1. 마지막으로 이 점수들을 평균하여 최종 검증 점수를 얻음.

- k-폴드(겹) 교차 검증(k-fold cross validation): 훈련 세트를 k만큼 부분으로 나눠서 교차 검증을 수행하는 방법. 보통 5나 10으로 나누는 5-폴드 교차 검증, 10-폴드 교차 검증을 많이 사용함. 이렇게 하면 데이터의 80 ~ 90%까지 훈련에 사용할 수 있음. 검증 세트는 줄지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 판단할 수 있음.

- 3-폴드 교차 검증을 예로 들면 아래처럼 훈련 세트를 세 부분으로 나누고, 훈련 세트로는 모델을 훈련하고, 검증 세트로는 모델을 평가함.

 1. 훈련 세트, 훈련 세트, 검증 세트
 
 1. 훈련 세트, 검증 세트, 훈련 세트
 
 1. 검증 세트, 훈련 세트, 훈련 세트

In [5]:
# 'cross_validate()' 함수를 사용하여 교차 검증을 실시함. 
# 먼저 평가할 모델 객체(결정 트리를 의미하는 dt)를 첫 번째 매개변수로 전달하고,
# 직접 검증 세트를 떼지 않고 훈련 세트 전체를 두 세 번째 매개변수에 전달함.
# 'cv' 매개변수에서 폴드 수를 지정함. 기본값은 5.
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
# 'fit_time', 'score_time', 'test_score' 키를 가진 딕셔너리를 반환함.
# fit_time: 훈련하는 시간.
# score_time: 검증하는 시간.
# test_score: 검증 폴드의 점수. 출력된 점수 5개를 평균하여 교차 검증의 최종 점수를 얻을 수 있음.

{'fit_time': array([0.00693226, 0.00658846, 0.00682378, 0.0067122 , 0.00648093]), 'score_time': array([0.0007329 , 0.00069785, 0.00062466, 0.00063729, 0.00064087]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


### 최상의 검증 점수 확인하기

In [6]:
# 'test_score' 의 점수 5개를 평균하여 교차 검증의 최상의 점수를 확인함.
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


### 분할기로 폴드 나누기

In [9]:
# 위에서 'train_test_split()' 함수로 전체 데이터를 섞었지만
# 본래 'cross_validate()' 함수는 훈련 세트를 섞고 폴드를 나누지 않음.
# 따라서 교차 검증을 할 때 훈련 세트를 섞으려면 'cv' 매개변수에 '분할기(splitter)' 를 지정해야 함.
# 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정하며,
# 회귀 모델일 경우 'KFold분할기' 를 사용하고,
# 분류 모델일 경우 'StratifiedKFold' 를 사용함.
# 즉 앞서 수행한 교차 검증은 아래처럼 표현함.
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 훈련 세트를 섞은 후 '10-폴드 교차 검증' 을 수행함.
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
# n_splits: 훈련 세트를 몇 부분으로 나눌지 결정하는 매개변수. 몇(k) 폴드 교차 검증을 할지 결정함. 기본값은 5.
# shuffle: 세트의 샘플을 섞을지 결정하는 매개변수. 기본값은 False.

scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))
# 'KFold' 클래스도 동일한 방식으로 사용함.
# 5-폴드와 10-폴드의 차이가 크지 않음.

0.8574181117533719


In [ ]:
정리 중..